In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

2024-10-07 15:25:15.047884: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-07 15:25:15.743846: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-07 15:25:15.747952: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-07 15:25:17.865623: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()

In [3]:
x_train.shape,y_train.shape,x_test.shape,y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [4]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [5]:
x_train=x_train.astype('float32')/255.0
x_test=x_test.astype('float32')/255.0

In [6]:
y_train=to_categorical(y_train,28)
y_test=to_categorical(y_test,28)

In [7]:
y_train.shape

(60000, 28)

In [8]:
base_model=tf.keras.applications.MobileNetV2(input_shape=(224,224,3),include_top=False,weights='imagenet')

2024-10-07 15:25:22.045986: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [9]:
model=models.Sequential()
model.add(layers.Reshape((28,28,1),input_shape=(28,28)))
model.add(layers.Conv2D(32,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(28,activation='softmax'))

In [10]:
for layer in base_model.layers:
    layer.trainable=False

In [11]:
model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy']
             )

In [12]:
epochs=10
batch_size=32

train_dataset=tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(6000).batch(batch_size)
val_dataset=tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(batch_size)

2024-10-07 15:25:23.543315: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


In [13]:
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    # Training loop
    for images, labels in train_dataset:
        with tf.GradientTape() as tape:
            predictions = model(images)
            loss = tf.keras.losses.categorical_crossentropy(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer = optimizers.Adam(learning_rate=0.001)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # Validation loop
    accuracy = tf.metrics.CategoricalAccuracy()
    for images, labels in val_dataset:
        predictions = model(images)
        accuracy.update_state(labels, predictions)
    val_acc = accuracy.result().numpy()
    print(f"Validation Accuracy: {val_acc}")

2024-10-07 15:25:23.808929: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


Epoch 1/10
Validation Accuracy: 0.9675999879837036
Epoch 2/10
Validation Accuracy: 0.9692999720573425
Epoch 3/10
Validation Accuracy: 0.9603999853134155
Epoch 4/10
Validation Accuracy: 0.9664000272750854
Epoch 5/10
Validation Accuracy: 0.9700999855995178
Epoch 6/10
Validation Accuracy: 0.9781000018119812
Epoch 7/10
Validation Accuracy: 0.978600025177002
Epoch 8/10
Validation Accuracy: 0.9746000170707703
Epoch 9/10
Validation Accuracy: 0.9800999760627747
Epoch 10/10
Validation Accuracy: 0.9714000225067139


In [14]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.5509 - accuracy: 0.9714


[0.5509325265884399, 0.9714000225067139]